# Loading Crypto Data

In [41]:
import numpy as np

import os

import pickle

import yfinance as yf

In [42]:
save_path = '../data/crypto/'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [4]:
tickers = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'SOL-USD']
names = ['Bitcoin', 'Ethereum', 'Tether', 'BNB', 'Solana']

In [2]:
data = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = tickers,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            #period = "1mo",

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            #interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            #group_by = 'Ticker', #Ticker

            # adjust all OHLC automatically
            # (optional, default is False)
            #auto_adjust = False,

            # download pre/post regular market hours data
            # (optional, default is False)
            #prepost = False,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            #threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            #proxy = None
        )

[*********************100%%**********************]  5 of 5 completed


In [45]:
vix_data = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = ['^VIX'],

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            #period = "1mo",

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            #interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            #group_by = 'Ticker', #Ticker

            # adjust all OHLC automatically
            # (optional, default is False)
            #auto_adjust = False,

            # download pre/post regular market hours data
            # (optional, default is False)
            #prepost = False,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            #threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            #proxy = None
        )

[*********************100%%**********************]  1 of 1 completed


In [47]:
vix = vix_data['Adj Close']

In [49]:
ln_vix = np.log(vix).diff()

In [54]:
ln_vix

Date
1990-01-02         NaN
1990-01-03    0.053640
1990-01-04    0.055079
1990-01-05    0.045266
1990-01-08    0.007431
                ...   
2024-05-09   -0.024135
2024-05-10   -0.011094
2024-05-13    0.080349
2024-05-14   -0.013324
2024-05-15   -0.075026
Name: Adj Close, Length: 8659, dtype: float64

In [13]:
adj_close = data['Adj Close']

In [14]:
adj_close = adj_close.dropna(inplace=False)

In [20]:
ln_diff = np.log(adj_close).diff()

In [28]:
cov_val = (100*ln_diff).rolling(21).cov().unstack()

In [52]:
cov_val

BNB-USD                                               BTC-USD  \
              BNB-USD    BTC-USD    ETH-USD    SOL-USD  USDT-USD    BNB-USD   
Date                                                                          
2020-05-01  13.235768   9.404082  12.763614  15.023982 -0.243764   9.404082   
2020-05-02  13.267711   9.351383  12.759285  14.416249 -0.228011   9.351383   
2020-05-03  13.460919   9.698847  13.313856  13.139171 -0.381238   9.698847   
2020-05-04  12.791801  10.314742  14.254221  15.362819 -0.245070  10.314742   
2020-05-05  12.551412  10.514398  14.478253  17.756269 -0.151238  10.514398   
...               ...        ...        ...        ...       ...        ...   
2024-05-11   4.103072   3.891253   3.149461   3.049296  0.030387   3.891253   
2024-05-12   4.014057   3.896167   3.159218   3.138023  0.031433   3.896167   
2024-05-13   3.068013   3.138463   2.678580   1.812631  0.027395   3.138463   
2024-05-14   3.891829   3.521252   3.040163   2.505295  0.032776   3.521252   
2024-05-15   4.532669   4.927780   4.005008   4.347912  0.049064   4.927780   

                                                       ...    SOL-USD  \
              BTC-USD    ETH-USD    SOL-USD  USDT-USD  ...    BNB-USD   
Date                                                   ...              
2020-05-01  12.262446  12.858868  18.563615  0.129833  ...  15.023982   
2020-05-02  12.172811  12.746475  17.251270  0.106453  ...  14.416249   
2020-05-03  12.416715  13.122871  17.475971  0.086548  ...  13.139171   
2020-05-04  12.010365  12.594329  15.751862 -0.045635  ...  15.362819   
2020-05-05  11.926961  12.589275  14.763236 -0.062999  ...  17.756269   
...               ...        ...        ...       ...  ...        ...   
2024-05-11   6.768052   5.073381   6.428860  0.066592  ...   3.049296   
2024-05-12   6.858985   5.137711   6.370345  0.065663  ...   3.138023   
2024-05-13   6.695802   4.947196   5.826178  0.067423  ...   1.812631   
2024-05-14   6.860332   5.136810   6.109561  0.069288  ...   2.505295   
2024-05-15   9.099763   6.672750   8.960772  0.096652  ...   4.347912   

                                                        USDT-USD            \
              BTC-USD    ETH-USD     SOL-USD  USDT-USD   BNB-USD   BTC-USD   
Date                                                                         
2020-05-01  18.563615  27.418684  115.014816  1.521527 -0.243764  0.129833   
2020-05-02  17.251270  25.933914   96.227270  1.256025 -0.228011  0.106453   
2020-05-03  17.475971  26.377856   87.981231  0.576813 -0.381238  0.086548   
2020-05-04  15.751862  23.930099   81.239947  0.110534 -0.245070 -0.045635   
2020-05-05  14.763236  23.894136   69.524327 -0.089382 -0.151238 -0.062999   
...               ...        ...         ...       ...       ...       ...   
2024-05-11   6.428860   6.114429   16.586913  0.052641  0.030387  0.066592   
2024-05-12   6.370345   6.066569   16.529994  0.052111  0.031433  0.065663   
2024-05-13   5.826178   5.619933   15.276432  0.052277  0.027395  0.067423   
2024-05-14   6.109561   5.986272   15.747860  0.054968  0.032776  0.069288   
2024-05-15   8.960772   7.941775   19.366899  0.089999  0.049064  0.096652   

                                          
             ETH-USD   SOL-USD  USDT-USD  
Date                                      
2020-05-01  0.016595  1.521527  0.263320  
2020-05-02  0.016226  1.256025  0.271050  
2020-05-03 -0.006894  0.576813  0.221006  
2020-05-04 -0.212631  0.110534  0.193587  
2020-05-05 -0.167055 -0.089382  0.201472  
...              ...       ...       ...  
2024-05-11  0.042511  0.052641  0.002023  
2024-05-12  0.041776  0.052111  0.002019  
2024-05-13  0.041913  0.052277  0.002058  
2024-05-14  0.045305  0.054968  0.002063  
2024-05-15  0.064074  0.089999  0.002394  

[1476 rows x 25 columns]

In [43]:
data_dict = {'adj_close': adj_close,
             'ln_diff': ln_diff,
             'cov': cov_val,
             'tickers': tickers,
             'names': names
            }

In [44]:
with open(''.join((save_path, 'crypto.pkl')), 'wb') as f:
    pickle.dump(data_dict, f)